# Fine-Tuning for SQL to Text

In [1]:
import os
import torch
import time
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from transformers import TextDataset, T5ForConditionalGeneration
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, concatenate_datasets

2024-10-09 08:21:23.168657: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import datasets
import transformers
print(datasets.__version__)
print(transformers.__version__)

3.0.1
4.44.0


Define config parameters

In [2]:
# model_name='google/flan-t5-small'
model_name='t5-small'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'    

Create model

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
base_model = base_model.to(device)

Load integrated datasets

In [2]:
dataset_csql_train = load_dataset("b-mc2/sql-create-context", split='train[:8000]')
dataset_csql_test = load_dataset("b-mc2/sql-create-context", split='train[-2000:-1000]')
dataset_csql_validation = load_dataset("b-mc2/sql-create-context", split='train[-1000:]')

dataset_tsql_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:8000]')
dataset_tsql_train = dataset_tsql_train.remove_columns(['source', 'text'])
dataset_tsql_train = dataset_tsql_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

dataset_tsql_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-2000:-1000]')
dataset_tsql_test  = dataset_tsql_test.remove_columns(['source', 'text'])
dataset_tsql_test  = dataset_tsql_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

dataset_tsql_validation   = load_dataset("Clinton/Text-to-sql-v1", split='train[-1000:]')
dataset_tsql_validation   = dataset_tsql_validation.remove_columns(['source', 'text'])
dataset_tsql_validation   = dataset_tsql_validation.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

In [4]:
dataset_local_train = load_dataset('csv', split='train[:80%]', data_files={'local_merged.csv'})
dataset_local_train

Generating train split: 0 examples [00:00, ? examples/s]

/Users/johnmoses/miniforge3/envs/mforge39/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 1600
})

In [5]:
dataset_local_test = load_dataset('csv', split='train[-20%:-10%]', data_files={'local_merged.csv'})
dataset_local_test

Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 200
})

In [6]:
dataset_local_validation = load_dataset('csv', split='train[-10%:]', data_files={'local_merged.csv'})
dataset_local_validation

Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 200
})

In [8]:
# dataset_x = DatasetDict({
#     'train': interleave_datasets([dataset_tsql_train, dataset_local_train]),
#     'test': interleave_datasets([dataset_tsql_test, dataset_local_test]),
#     'validation': interleave_datasets([dataset_tsql_validation, dataset_local_validation])
# })

In [9]:
# dataset_x

In [7]:
dataset_train_merged = concatenate_datasets(
    [
        dataset_csql_train, 
        dataset_tsql_train,
        dataset_local_train
        ]
    )
dataset_test_merged = concatenate_datasets(
    [
        dataset_csql_test, 
        dataset_tsql_test,
        dataset_local_test
        ]
    )
dataset_validation_merged = concatenate_datasets(
    [
        dataset_csql_validation, 
        dataset_tsql_validation,
        dataset_local_validation
        ]
    )

Save datasets in `.csv` format

In [8]:
dataset_train_merged.to_csv('train_merged.csv', index=False)
dataset_test_merged.to_csv('test_merged.csv', index=False)
dataset_validation_merged.to_csv('validation_merged.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1293725

In [9]:
dataset = load_dataset('csv', data_files={
    "train": "train_merged.csv", 
    "test": "test_merged.csv", 
    "validation": "validation_merged.csv"
    })
dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 17600
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 2200
    })
    validation: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 2200
    })
})

View sample

In [6]:
dataset['test'][0]

{'context': 'CREATE TABLE table_name_94 (round VARCHAR, event VARCHAR)',
 'question': 'Which round has pain and glory 2006 as the event?',
 'answer': 'SELECT round FROM table_name_94 WHERE event = "pain and glory 2006"'}

Explore dataset

In [10]:
dataset_train = pd.read_csv('./train_merged.csv')
dataset_train.head()

,context,question,answer
0,CREATE TABLE head (age INTEGER),How many heads of the departments are older th...,SELECT COUNT(*) FROM head WHERE age > 56
1,"CREATE TABLE head (name VARCHAR, born_state VA...","List the name, born state and age of the heads...","SELECT name, born_state, age FROM head ORDER B..."
2,"CREATE TABLE department (creation VARCHAR, nam...","List the creation year, name and budget of eac...","SELECT creation, name, budget_in_billions FROM..."
3,CREATE TABLE department (budget_in_billions IN...,What are the maximum and minimum budget of the...,"SELECT MAX(budget_in_billions), MIN(budget_in_..."
4,CREATE TABLE department (num_employees INTEGER...,What is the average number of employees of the...,SELECT AVG(num_employees) FROM department WHER...


In [47]:
# CHAT_ML_TEMPLATE = """
# {% for message in messages %}
#     {% if message['role'] == 'user' %}
#         {{'<|im_start|>user\n' + message['content'].strip() + '<|im_end|>' }}
#     {% elif message['role'] == 'system' %}
#         {{'<|im_start|>system\n' + message['content'].strip() + '<|im_end|>' }}
#     {% elif message['role'] == 'assistant' %}
#         {{'<|im_start|>assistant\n'  + message['content'] + '<|im_end|>' }}
#     {% endif %}
# {% endfor %}
# """

# ASSISTANT_PROMPT = "<|im_start|>assistant\n"

# EOS_TOKEN = "<|im_end|>"

Create tokenizer

In [11]:
# tokenizer = AutoTokenizer.from_pretrained(model_name, eos_token="<|im_end|>")
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.chat_template = CHAT_ML_TEMPLATE

In [12]:
def tokenize_function(example):
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"

    data_zip = zip(example['context'], example['question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return example

In [13]:
shuffled_dataset = dataset.shuffle(seed=42)

In [15]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['question','context','answer'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 17600
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2200
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2200
    })
})

Test with zero shot prediction

In [16]:
index = 0

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cpu')

output = tokenizer.decode(
    base_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_94 (round VARCHAR, event VARCHAR)

Question:
Which round has pain and glory 2006 as the event?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT round FROM table_name_94 WHERE event = "pain and glory 2006"

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Question: Which round has pain and glory 2006 as the event? Answer: Which round has pain and glory 2006 as the event?


In [17]:
def translate_to_sql(query):
    input_text = "translate English to SQL: %s " % query
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = base_model.generate(input_ids)
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

In [18]:
question = "Which round has pain and glory 2006 as the event?"
output = translate_to_sql(question)
print("SQL Query:", output)

SQL Query: Welche Runde hat Schmerz und Ruhm 2006 als Veranstaltung?


/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [19]:
def get_sql(query):
    input_text = f'translate English to SQL: {query}'
    features = tokenizer([input_text], return_tensors='pt').to(device)
    output = base_model.generate(
        input_ids=features['input_ids'],
        attention_mask=features['attention_mask'],
        max_new_tokens=200
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [20]:
query = "Which round has pain and glory 2006 as the event?"
get_sql(query)

'Welche Runde hat Schmerz und Ruhm 2006 als Veranstaltung?'

In [21]:
for i in range(0,5):
    print('Question: ' + dataset['test'][i]['question'])
    print('Predict. :' + get_sql(dataset['test'][i]['question']))
    print('Expected: ' + dataset['test'][i]['answer'])
    print('=================================\n')

Question: Which round has pain and glory 2006 as the event?
Predict. :Welche Runde hat Schmerz und Ruhm 2006 als Veranstaltung?
Expected: SELECT round FROM table_name_94 WHERE event = "pain and glory 2006"

Question: Which record has john flemming as the opponent?
Predict. :SQL-Record hat john flemming als Gegner?
Expected: SELECT record FROM table_name_14 WHERE opponent = "john flemming"

Question: Which record has 5:00 as the time, jess liaudin as the opponent for the location of england?
Predict. :SQL hat 5:00 als Zeit, jess liaudin als Gegner für die Lage der england?
Expected: SELECT record FROM table_name_49 WHERE time = "5:00" AND opponent = "jess liaudin" AND location = "england"

Question: Name the score for home of green bay packers
Predict. :SQL - Name the score for home of green bay packers
Expected: SELECT score FROM table_name_37 WHERE home = "green bay packers"

Question: Name the home for 21-17
Predict. :Identifizieren Sie die Heimat für 21-17
Expected: SELECT home FROM

Fine tune

In [19]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
finetuned_model = finetuned_model.to('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [27]:
output_dir = 'training-t5'

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    learning_rate=5e-3,
    num_train_epochs=2,
    per_device_train_batch_size=16,     # batch size per device during training
    per_device_eval_batch_size=16,      # batch size for evaluation
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',        # evaluation strategy to adopt during training
    eval_steps=500,                  
)

trainer = Trainer(
    model=finetuned_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
trainer.evaluate()

  0%|          | 0/138 [00:00<?, ?it/s]

{'eval_loss': 1.4413552284240723,
 'eval_model_preparation_time': 0.002,
 'eval_runtime': 84.4542,
 'eval_samples_per_second': 26.05,
 'eval_steps_per_second': 1.634}

In [29]:
%%time

trainer.train()

  0%|          | 0/2200 [00:00<?, ?it/s]

{'loss': 0.418, 'grad_norm': 0.14278097450733185, 'learning_rate': 0.004886363636363637, 'epoch': 0.05}
{'loss': 0.1458, 'grad_norm': 0.10544981807470322, 'learning_rate': 0.004772727272727273, 'epoch': 0.09}
{'loss': 0.1129, 'grad_norm': 0.07779989391565323, 'learning_rate': 0.004659090909090909, 'epoch': 0.14}
{'loss': 0.1045, 'grad_norm': 0.06239340826869011, 'learning_rate': 0.004545454545454545, 'epoch': 0.18}
{'loss': 0.0967, 'grad_norm': 0.07310223579406738, 'learning_rate': 0.004431818181818182, 'epoch': 0.23}
{'loss': 0.0687, 'grad_norm': 0.07015125453472137, 'learning_rate': 0.004318181818181818, 'epoch': 0.27}
{'loss': 0.062, 'grad_norm': 0.1204940602183342, 'learning_rate': 0.004204545454545455, 'epoch': 0.32}
{'loss': 0.0638, 'grad_norm': 0.05975992605090141, 'learning_rate': 0.004090909090909091, 'epoch': 0.36}
{'loss': 0.0601, 'grad_norm': 0.06554871797561646, 'learning_rate': 0.003977272727272727, 'epoch': 0.41}
{'loss': 0.061, 'grad_norm': 0.04516300559043884, 'learnin

  0%|          | 0/138 [00:00<?, ?it/s]

{'eval_loss': 0.04569010064005852, 'eval_model_preparation_time': 0.002, 'eval_runtime': 84.6068, 'eval_samples_per_second': 26.003, 'eval_steps_per_second': 1.631, 'epoch': 0.45}
{'loss': 0.0604, 'grad_norm': 0.06717697530984879, 'learning_rate': 0.00375, 'epoch': 0.5}
{'loss': 0.0546, 'grad_norm': 0.08848704397678375, 'learning_rate': 0.0036363636363636364, 'epoch': 0.55}
{'loss': 0.0562, 'grad_norm': 0.05489804595708847, 'learning_rate': 0.003522727272727273, 'epoch': 0.59}
{'loss': 0.0603, 'grad_norm': 0.060955531895160675, 'learning_rate': 0.003409090909090909, 'epoch': 0.64}
{'loss': 0.0538, 'grad_norm': 0.06508537381887436, 'learning_rate': 0.0032954545454545454, 'epoch': 0.68}
{'loss': 0.0417, 'grad_norm': 0.07296702265739441, 'learning_rate': 0.003181818181818182, 'epoch': 0.73}
{'loss': 0.0476, 'grad_norm': 0.062193285673856735, 'learning_rate': 0.0030681818181818184, 'epoch': 0.77}
{'loss': 0.0449, 'grad_norm': 0.06822644174098969, 'learning_rate': 0.002954545454545455, 'epo

  0%|          | 0/138 [00:00<?, ?it/s]

...

  0%|          | 0/138 [00:00<?, ?it/s]

{'eval_loss': 0.02869999222457409, 'eval_model_preparation_time': 0.002, 'eval_runtime': 84.7172, 'eval_samples_per_second': 25.969, 'eval_steps_per_second': 1.629, 'epoch': 1.36}
{'loss': 0.0284, 'grad_norm': 0.05072499439120293, 'learning_rate': 0.0014772727272727275, 'epoch': 1.41}
{'loss': 0.035, 'grad_norm': 0.043536391109228134, 'learning_rate': 0.0013636363636363635, 'epoch': 1.45}
{'loss': 0.0317, 'grad_norm': 0.033162590116262436, 'learning_rate': 0.00125, 'epoch': 1.5}
{'loss': 0.03, 'grad_norm': 0.0664527639746666, 'learning_rate': 0.0011363636363636363, 'epoch': 1.55}
{'loss': 0.0339, 'grad_norm': 0.03389521688222885, 'learning_rate': 0.0010227272727272728, 'epoch': 1.59}
{'loss': 0.0254, 'grad_norm': 0.051419828087091446, 'learning_rate': 0.0009090909090909091, 'epoch': 1.64}
{'loss': 0.0276, 'grad_norm': 0.054062169045209885, 'learning_rate': 0.0007954545454545455, 'epoch': 1.68}
{'loss': 0.0256, 'grad_norm': 0.06520362943410873, 'learning_rate': 0.0006818181818181818, 'e

  0%|          | 0/138 [00:00<?, ?it/s]

{'eval_loss': 0.024220574647188187, 'eval_model_preparation_time': 0.002, 'eval_runtime': 84.8041, 'eval_samples_per_second': 25.942, 'eval_steps_per_second': 1.627, 'epoch': 1.82}
{'loss': 0.0238, 'grad_norm': 0.059269968420267105, 'learning_rate': 0.0003409090909090909, 'epoch': 1.86}
{'loss': 0.0329, 'grad_norm': 0.036003947257995605, 'learning_rate': 0.00022727272727272727, 'epoch': 1.91}
{'loss': 0.0222, 'grad_norm': 0.0551101379096508, 'learning_rate': 0.00011363636363636364, 'epoch': 1.95}
{'loss': 0.0273, 'grad_norm': 0.11864445358514786, 'learning_rate': 0.0, 'epoch': 2.0}
{'train_runtime': 7985.0875, 'train_samples_per_second': 4.408, 'train_steps_per_second': 0.276, 'train_loss': 0.05568295933983543, 'epoch': 2.0}
CPU times: user 23min 13s, sys: 11min 52s, total: 35min 6s
Wall time: 2h 13min 5s


TrainOutput(global_step=2200, training_loss=0.05568295933983543, metrics={'train_runtime': 7985.0875, 'train_samples_per_second': 4.408, 'train_steps_per_second': 0.276, 'total_flos': 4764031411814400.0, 'train_loss': 0.05568295933983543, 'epoch': 2.0})

In [30]:
finetuned_model.save_pretrained("model-t5")

In [31]:
tokenizer.save_pretrained("model-t5")

('model-t5/tokenizer_config.json',
 'model-t5/special_tokens_map.json',
 'model-t5/tokenizer.json')

In [22]:
model_path = "model-t5"
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
finetuned_model = finetuned_model.to(device)

finetuned_tokenizer = AutoTokenizer.from_pretrained(model_path)

Test fine-tuned model with zero shot inferencing

In [23]:
index = 0
# index = len(dataset['test'])-100

question = dataset['test'][index]['question']
context = dataset['test'][index]['context']
answer = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to('cpu')

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Tables:
CREATE TABLE table_name_94 (round VARCHAR, event VARCHAR)

Question:
Which round has pain and glory 2006 as the event?

Answer:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
SELECT round FROM table_name_94 WHERE event = "pain and glory 2006"

---------------------------------------------------------------------------------------------------
FINE-TUNED MODEL - ZERO SHOT:
SELECT round FROM table_name_94 WHERE event = "pain and glory 2006"


Test with query

In [24]:
def get_sql(query):
    prompt = "translate English to SQL: %s " % query
    features = tokenizer([prompt], return_tensors='pt')
    output = finetuned_model.generate(
        input_ids=features['input_ids'],
        max_new_tokens=200
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [25]:
query = "What is my user id?"
get_sql(query)

'SELECT id FROM id WHERE SQL = "johnmoses"'

In [26]:
for i in range(0,10, 1):
    print('Question: ' + dataset['test'][i]['question'])
    print('Predict. :' + get_sql(dataset['test'][i]['question']))
    print('Expected: ' + dataset['test'][i]['answer'])
    print('=================================\n')

Question: Which round has pain and glory 2006 as the event?
Predict. :SELECT round FROM round WHERE event = "Purple and glory 2006"
Expected: SELECT round FROM table_name_94 WHERE event = "pain and glory 2006"

Question: Which record has john flemming as the opponent?
Predict. :SELECT record FROM record WHERE opponent = "John flemming"
Expected: SELECT record FROM table_name_14 WHERE opponent = "john flemming"

Question: Which record has 5:00 as the time, jess liaudin as the opponent for the location of england?
Predict. :SELECT record FROM record WHERE location = 5:00 AND location = 'england'
Expected: SELECT record FROM table_name_49 WHERE time = "5:00" AND opponent = "jess liaudin" AND location = "england"

Question: Name the score for home of green bay packers
Predict. :SELECT score FROM score WHERE home of green bay packers.yard_name = "Home of Green Bay Packers"
Expected: SELECT score FROM table_name_37 WHERE home = "green bay packers"

Question: Name the home for 21-17
Predict. 

PEFT

In [27]:
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
from safetensors.torch import save_model

In [27]:
peft_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
peft_model = peft_model.to('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [30]:
output_dir = 'training-peft-t5'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    learning_rate=1e-3,
    num_train_epochs= 3
)

peft_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
    
peft_trainer = SFTTrainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset= tokenized_datasets['train'],
    eval_dataset= tokenized_datasets['test'],
    dataset_text_field="text",
    tokenizer=tokenizer,
    packing=True,
    peft_config=peft_config,
    max_seq_length=tokenizer.model_max_length,
)

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [38]:
peft_trainer.evaluate()

  0%|          | 0/440 [00:00<?, ?it/s]

{'eval_loss': 1.5420283079147339,
 'eval_model_preparation_time': 0.004,
 'eval_runtime': 93.2244,
 'eval_samples_per_second': 23.599,
 'eval_steps_per_second': 4.72}

In [39]:
%%time
peft_trainer.train()

  0%|          | 0/10560 [00:00<?, ?it/s]

{'loss': 0.2879, 'grad_norm': 0.12263398617506027, 'learning_rate': 0.0009526515151515152, 'epoch': 0.14}
{'loss': 0.1362, 'grad_norm': 0.10611046105623245, 'learning_rate': 0.0009053030303030303, 'epoch': 0.28}
{'loss': 0.097, 'grad_norm': 0.1095716655254364, 'learning_rate': 0.0008579545454545454, 'epoch': 0.43}
{'loss': 0.0925, 'grad_norm': 0.10936679691076279, 'learning_rate': 0.0008106060606060606, 'epoch': 0.57}


/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d2035bba-0413-448d-9603-8159d86164ff)') - silently ignoring the lookup for the file config.json in t5-small.
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in t5-small - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.0862, 'grad_norm': 0.081715889275074, 'learning_rate': 0.0007632575757575758, 'epoch': 0.71}
{'loss': 0.0759, 'grad_norm': 0.10686590522527695, 'learning_rate': 0.0007159090909090909, 'epoch': 0.85}
{'loss': 0.0677, 'grad_norm': 0.04453447088599205, 'learning_rate': 0.0006685606060606061, 'epoch': 0.99}
{'loss': 0.0693, 'grad_norm': 0.15954449772834778, 'learning_rate': 0.0006212121212121212, 'epoch': 1.14}
{'loss': 0.066, 'grad_norm': 0.06855412572622299, 'learning_rate': 0.0005738636363636364, 'epoch': 1.28}


/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 18feed48-4139-4a31-b924-0129e95152d2)') - silently ignoring the lookup for the file config.json in t5-small.
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in t5-small - will assume that the vocabulary was not modified.
  warnings.warn(


...

/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /t5-small/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f84b8d81f10>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 0626c8d9-48ec-429c-b4cd-32d20f190354)') - silently ignoring the lookup for the file config.json in t5-small.
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in t5-small - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.0489, 'grad_norm': 0.07677163183689117, 'learning_rate': 0.00010037878787878788, 'epoch': 2.7}


/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/other.py:619: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /t5-small/resolve/main/config.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f8418a49340>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 775c614f-621f-4072-8d36-e059e4c43856)') - silently ignoring the lookup for the file config.json in t5-small.
  warnings.warn(
/Users/johnmoses/miniconda3/envs/mconda38/lib/python3.8/site-packages/peft/utils/save_and_load.py:218: UserWarning: Could not find a config file in t5-small - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.0466, 'grad_norm': 0.05929362773895264, 'learning_rate': 5.303030303030303e-05, 'epoch': 2.84}
{'loss': 0.0545, 'grad_norm': 0.05964748188853264, 'learning_rate': 5.681818181818182e-06, 'epoch': 2.98}
{'train_runtime': 21606.7415, 'train_samples_per_second': 2.444, 'train_steps_per_second': 0.489, 'train_loss': 0.07824968086047605, 'epoch': 3.0}
CPU times: user 1h 12min 2s, sys: 5min 54s, total: 1h 17min 56s
Wall time: 6h 6s


TrainOutput(global_step=10560, training_loss=0.07824968086047605, metrics={'train_runtime': 21606.7415, 'train_samples_per_second': 2.444, 'train_steps_per_second': 0.489, 'total_flos': 7337387910758400.0, 'train_loss': 0.07824968086047605, 'epoch': 3.0})

In [40]:
peft_model.save_pretrained("model-peft-t5")
tokenizer.save_pretrained("model-peft-t5")

('model-peft-t5/tokenizer_config.json',
 'model-peft-t5/special_tokens_map.json',
 'model-peft-t5/tokenizer.json')

In [44]:
# save_model(peft_model, "peft_model.safetensors")

In [33]:
peft_model = get_peft_model(base_model, peft_config)
print(peft_model.print_trainable_parameters())

trainable params: 1,179,648 || all params: 61,686,272 || trainable%: 1.9123
None


In [32]:
peft_model_path = "./model-peft-t5"
peft_model = AutoModelForSeq2SeqLM.from_pretrained(peft_model_path)

peft_tokenizer = AutoTokenizer.from_pretrained(peft_model_path)

Some weights of the model checkpoint at ./model-peft-t5 were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.0.SelfAttention.q.base_layer.weight', 'decoder.block.0.layer.0.SelfAttention.q.lora_A.default.weight', 'decoder.block.0.layer.0.SelfAttention.q.lora_B.default.weight', 'decoder.block.0.layer.0.SelfAttention.v.base_layer.weight', 'decoder.block.0.layer.0.SelfAttention.v.lora_A.default.weight', 'decoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight', 'decoder.block.0.layer.1.EncDecAttention.q.base_layer.weight', 'decoder.block.0.layer.1.EncDecAttention.q.lora_A.default.weight', 'decoder.block.0.layer.1.EncDecAttention.q.lora_B.default.weight', 'decoder.block.0.layer.1.EncDecAttention.v.base_layer.weight', 'decoder.block.0.layer.1.EncDecAttention.v.lora_A.default.weight', 'decoder.block.0.layer.1.EncDecAttention.v.lora_B.default.weight', 'decoder.block.1.layer.0.SelfAttention.q.base_layer.weight', 'decoder.block.1.layer.0.SelfAttention.q.lora_

In [34]:
def get_peft_sql(query):
    prompt = "translate English to SQL: %s </s>" % query
    features = tokenizer([prompt], return_tensors='pt')
    output = peft_model.generate(
        input_ids=features['input_ids'],
        attention_mask=features["attention_mask"],
        max_new_tokens=200
        )
    return tokenizer.decode(output[0], skip_special_tokens=True).replace("<pad> ", "", 1).replace("</s>", "")

In [35]:
query = "How many meetings do we have?"
get_peft_sql(query)

'Wie viele Treffen haben wir?'

Evaluate quantitatively with ROUGE Metric
Perform inferences for test dataset. Do 5 only, due to time it takes.

In [36]:
import evaluate

In [37]:
questions = dataset['test'][0:20]['question']
contexts = dataset['test'][0:20]['context']
human_answers = dataset['test'][0:20]['answer']

base_model_answers = []
finetuned_model_answers = []
peft_model_answers = []

for idx, question in enumerate(questions):
    prompt = f"""Tables:
    {contexts[idx]}

    Question:
    {question}

    Answer:
    """
    
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    input_ids = input_ids.to(device)

    human_text_output = human_answers[idx]

    base_model_outputs = base_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    base_model_output = tokenizer.decode(base_model_outputs[0], skip_special_tokens=True)
    base_model_answers.append(base_model_output)

    finetuned_model_outputs = finetuned_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    finetuned_model_output = tokenizer.decode(finetuned_model_outputs[0], skip_special_tokens=True)
    finetuned_model_answers.append(finetuned_model_output)

    peft_model_outputs = peft_model.generate(input_ids=input_ids,generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
    peft_model_answers.append(peft_model_output)

In [38]:
zipped_summaries = list(zip(human_answers, base_model_answers, finetuned_model_answers, peft_model_answers))
df = pd.DataFrame(zipped_summaries, columns=['human_answers','base_model','finetuned_model','peft_model'])
df

,human_answers,base_model,finetuned_model,peft_model
0,SELECT round FROM table_name_94 WHERE event = ...,Question: Which round has pain and glory 2006 ...,SELECT round FROM table_name_94 WHERE event = ...,Question: Which round has pain and glory 2006 ...
1,SELECT record FROM table_name_14 WHERE opponen...,Question,SELECT record FROM table_name_14 WHERE opponen...,Question
2,SELECT record FROM table_name_49 WHERE time = ...,True,SELECT record FROM table_name_49 WHERE locatio...,True
3,"SELECT score FROM table_name_37 WHERE home = ""...",Tables: CREATE TABLE table_name_37 (score VARC...,"SELECT score FROM table_name_37 WHERE home = ""...",Tables: CREATE TABLE table_name_37 (score VARC...
4,"SELECT home FROM table_name_93 WHERE score = ""...",Tables: CREATE TABLE table_name_93 (home VARCH...,SELECT home FROM table_name_93 WHERE score = 2...,Tables: CREATE TABLE table_name_93 (home VARCH...
5,SELECT engine FROM table_name_85 WHERE entrant...,Question,SELECT engine FROM table_name_85 WHERE entrant...,Question
6,SELECT chassis FROM table_name_39 WHERE year <...,True,SELECT chassis FROM table_name_39 WHERE year ...,True
7,SELECT chassis FROM table_name_5 WHERE entrant...,Question,SELECT chassis FROM table_name_5 WHERE entrant...,Question
8,SELECT MIN(attendance) FROM table_name_3 WHERE...,Question,SELECT SUM(attendance) FROM table_name_3 WHERE...,Question
9,SELECT MIN(attendance) FROM table_name_7 WHERE...,Answer: How many people were in attendance on ...,SELECT SUM(attendance) FROM table_name_7 WHERE...,Answer: How many people were in attendance on ...


Compute ROUGE score for a subset of the data

In [39]:
# Load ROUGE
rouge = evaluate.load('rouge')

# Select dataset
answers = dataset['test'][0:20]['answer']

base_model_results = rouge.compute(
    predictions=base_model_answers,
    references=answers,
    use_aggregator=True,
    use_stemmer=True,
)
print('Base Model:\n',base_model_results)

finetuned_model_results = rouge.compute(
    predictions=finetuned_model_answers,
    references=answers,
    use_aggregator=True,
    use_stemmer=True,
)
print('Fine-tuned model:\n',finetuned_model_results)

peft_model_results = rouge.compute(
    predictions=peft_model_answers,
    references=answers,
    use_aggregator=True,
    use_stemmer=True,
)
print('PEFT model:\n',peft_model_results)

Base Model:
 {'rouge1': 0.06086309523809523, 'rouge2': 0.029091312056737588, 'rougeL': 0.05803571428571428, 'rougeLsum': 0.060057773109243685}
Fine-tuned model:
 {'rouge1': 0.9759720279720281, 'rouge2': 0.932786674844396, 'rougeL': 0.9612762237762238, 'rougeLsum': 0.9623723212271599}
PEFT model:
 {'rouge1': 0.06086309523809523, 'rouge2': 0.029091312056737588, 'rougeL': 0.05803571428571428, 'rougeLsum': 0.060057773109243685}
